In [1]:
import os
import glob
import re
import xarray as xr
import dask

def find_raster_files(root_dir):
    """
    Recursively search for .tif files in the root directory matching the pattern.
    Returns a dictionary of files categorized by (crop_type, year).
    """
    # Regular expression pattern to match the file names
    pattern = re.compile(r'.*_A_(\d{4})_(\w+)_A\.tif$')
    
    # Dictionary to store files by (crop_type, year)
    file_dict = {}

    # Walk through the root directory and subdirectories
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.tif'):
                match = pattern.search(file)
                if match:
                    year = match.group(1)
                    crop_type = match.group(2)
                    key = (crop_type, year)
                    if key not in file_dict:
                        file_dict[key] = []
                    file_path = os.path.join(root, file)
                    file_dict[key].append(file_path)
    
    return file_dict

def open_crops_by_year(file_dict):
    """
    Opens raster files by crop type and year using xarray.
    """
    datasets = {}

    for (crop_type, year), file_list in file_dict.items():
        try:
            # Use xarray to open multiple files as a single dataset
            ds = xr.open_mfdataset(file_list, combine='by_coords', chunks={'x': 1000, 'y': 1000}, engine='rasterio')
            datasets[(crop_type, year)] = ds
            print(f"Successfully loaded {crop_type} for {year}")
        except Exception as e:
            print(f"Failed to load {crop_type} for {year}: {e}")
    
    return datasets

# Usage example
root_directory = '/Users/mbraaksma/Files/base_data/spam'
raster_files = find_raster_files(root_directory)
crop_datasets = open_crops_by_year(raster_files)

# Access an example dataset for a specific crop and year
example_ds = crop_datasets.get(('maize', '2020'))
print(example_ds)


None


In [9]:
import glob
import re
import os
import xarray as xr
import dask

def find_raster_files(root_dir):
    """
    Recursively search for .tif files in the root directory matching the pattern.
    Returns a dictionary of files categorized by (crop_type, year).
    """
    # Regular expression pattern to match the file names
    pattern = re.compile(r'.*_A_(\d{4})_A\.tif$')
    
    # Dictionary to store files by (crop_type, year)
    file_dict = {}

    # Use glob to search for all .tif files recursively
    search_pattern = os.path.join(root_dir, '**', '*.tif')
    all_files = glob.glob(search_pattern, recursive=True)
    print(all_files)

    if not all_files:
        print("No .tif files found in the directory or its subdirectories.")
    
    for file_path in all_files:
        file_name = os.path.basename(file_path)
        match = pattern.search(file_name)
        if match:
            year = match.group(1)
            crop_type = match.group(2)
            key = (crop_type, year)
            if key not in file_dict:
                file_dict[key] = []
            file_dict[key].append(file_path)
    
    return file_dict

def open_crops_by_year(file_dict):
    """
    Opens raster files by crop type and year using xarray.
    """
    datasets = {}

    for (crop_type, year), file_list in file_dict.items():
        print(f"Loading {crop_type} data for {year}...")
        try:
            # Use xarray to open multiple files as a single dataset
            ds = xr.open_mfdataset(
                file_list, 
                combine='by_coords', 
                chunks={'x': 1000, 'y': 1000}, 
                engine='rasterio'
            )
            datasets[(crop_type, year)] = ds
            print(f"Successfully loaded {crop_type} for {year}")
        except Exception as e:
            print(f"Failed to load {crop_type} for {year}: {e}")
    
    return datasets

# Usage example
root_directory = '/Users/mbraaksma/Files/base_data/spam'
raster_files = find_raster_files(root_directory)
print(f"Found {len(raster_files)} crops/years with data.")

if raster_files:
    crop_datasets = open_crops_by_year(raster_files)

# Example access to one dataset
example_ds = crop_datasets.get(('maize', '2020'))
if example_ds:
    print(example_ds)
else:
    print("Dataset not found for maize in 2020.")


['/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_OCER_R.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_VEGE_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_TOBA_R.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_REST_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_SWPO_R.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_SUGC_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_ONIO_I.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_LENT_I.tif', '/Users

In [27]:
import glob
import re
import os
import xarray as xr

def find_raster_files(root_dir):
    """
    Recursively search for .tif files in the root directory matching the pattern.
    Extracts the year (four digits) and crop type (four capital letters).
    Returns a dictionary of files categorized by crop type.
    """
    # Pattern to identify the correct type of files
    pattern = re.compile(r'_A_[a-zA-Z]{4}_A.tif$')
    
    # Dictionary to store files by crop type
    file_dict = {}

    # Use glob to search for all .tif files recursively
    search_pattern = os.path.join(root_dir, '**', '*.tif')
    print(f"Searching for files with pattern: {search_pattern}")
    all_files = glob.glob(search_pattern, recursive=True)

    print(f"Number of .tif files found: {len(all_files)}")
    if not all_files:
        print("No .tif files found in the directory or its subdirectories.")
        return file_dict

    # Loop through each found file and apply the regex pattern
    for file_path in all_files:
        file_name = os.path.basename(file_path)
        
        # Check if the file matches the general pattern
        if pattern.search(file_name):
            
            # Extract the year (4 consecutive digits) and crop type (4 capital letters)
            year_match = re.search(r'(\d{4})', file_name)
            crop_type_match = re.search(r'_A_([A-Z]{4})_A', file_name)
            
            if year_match and crop_type_match:
                crop_type = crop_type_match.group(1)
                
                # Add the file to the list for the corresponding crop type
                if crop_type not in file_dict:
                    file_dict[crop_type] = []
                file_dict[crop_type].append(file_path)
                
    print(f"Total matched crops: {len(file_dict)}")
    return file_dict


def open_crops_by_year(file_dict):
    """
    Opens raster files by crop type using xarray.
    Combines yearly files into a single xarray dataset for each crop type.
    """
    datasets = {}

    for crop_type, file_list in file_dict.items():
        try:
            # Use xarray to open multiple files as a single dataset
            ds = xr.open_mfdataset(file_list, combine='by_coords', engine='rasterio')
            datasets[crop_type] = ds
            print(f"Successfully loaded dataset for crop: {crop_type}")
        except Exception as e:
            print(f"Failed to load dataset for crop: {crop_type} - Error: {e}")
    
    return datasets


# Usage example
root_directory = '/Users/mbraaksma/Files/base_data/spam'
raster_files = find_raster_files(root_directory)
crop_datasets = open_crops_by_year(raster_files)

# Example access to a dataset for a specific crop
if 'VEGE' in crop_datasets:
    example_ds = crop_datasets['VEGE']
    print(example_ds)
else:
    print("No dataset found for VEGE.")


Searching for files with pattern: /Users/mbraaksma/Files/base_data/spam/**/*.tif
Number of .tif files found: 3180
Total matched crops: 48
Failed to load dataset for crop: VEGE - Error: Resulting object does not have monotonic global indexes along dimension y
Failed to load dataset for crop: REST - Error: Resulting object does not have monotonic global indexes along dimension y
Failed to load dataset for crop: SUGC - Error: Resulting object does not have monotonic global indexes along dimension y
Failed to load dataset for crop: SORG - Error: Resulting object does not have monotonic global indexes along dimension y
Failed to load dataset for crop: MAIZ - Error: Resulting object does not have monotonic global indexes along dimension y
Failed to load dataset for crop: BANA - Error: Resulting object does not have monotonic global indexes along dimension y
Failed to load dataset for crop: SWPO - Error: Resulting object does not have monotonic global indexes along dimension y
Failed to load 

In [29]:
import glob
import re
import os
import xarray as xr

def find_raster_files(root_dir):
    """
    Recursively search for .tif files in the root directory matching the pattern.
    Extracts the year (four digits) and crop type (four capital letters).
    Returns a dictionary of files categorized by crop type.
    """
    # Pattern to identify the correct type of files
    pattern = re.compile(r'_A_[a-zA-Z]{4}_A.tif$')
    
    # Dictionary to store files by crop type
    file_dict = {}

    # Use glob to search for all .tif files recursively
    search_pattern = os.path.join(root_dir, '**', '*.tif')
    print(f"Searching for files with pattern: {search_pattern}")
    all_files = glob.glob(search_pattern, recursive=True)

    print(f"Number of .tif files found: {len(all_files)}")
    if not all_files:
        print("No .tif files found in the directory or its subdirectories.")
        return file_dict

    # Loop through each found file and apply the regex pattern
    for file_path in all_files:
        file_name = os.path.basename(file_path)
        
        # Check if the file matches the general pattern
        if pattern.search(file_name):
            
            # Extract the year (4 consecutive digits) and crop type (4 capital letters)
            year_match = re.search(r'(\d{4})', file_name)
            crop_type_match = re.search(r'_A_([A-Z]{4})_A', file_name)
            
            if year_match and crop_type_match:
                crop_type = crop_type_match.group(1)
                
                # Add the file to the list for the corresponding crop type
                if crop_type not in file_dict:
                    file_dict[crop_type] = []
                file_dict[crop_type].append(file_path)
                
    print(f"Total matched crops: {len(file_dict)}")
    return file_dict

def add_time_dim(xda):
    img_name = xda.encoding["source"]
    img_datetime = img_name.split("_")[-1].split("Z")[0]
    dt = datetime.strptime(img_datetime, "%Y-%m-%dT%H:%M:%S")
    dt = pd.to_datetime(dt)

    xda = xda.expand_dims(time = [dt])
    return xda

def open_crops_by_year(file_list):
    """
    Opens raster files by crop type using xarray.
    Combines yearly files into a single xarray dataset for each crop type.
    """

    ds = xr.open_mfdataset(file_list, combine='by_coords', engine='rasterio')
    # ds = xr.open_mfdataset(file_list, combine='by_coords', preprocess=add_time_dim, engine='rasterio')

    return ds


# Usage example
root_directory = '/Users/mbraaksma/Files/base_data/spam'
raster_files = find_raster_files(root_directory)


crop_dataset = open_crops_by_year(raster_files['VEGE'])
print(crop_dataset)


Searching for files with pattern: /Users/mbraaksma/Files/base_data/spam/**/*.tif
Number of .tif files found: 3180
Total matched crops: 48


ValueError: Resulting object does not have monotonic global indexes along dimension y

In [30]:
import glob
import re
import os
import xarray as xr
import numpy as np

def find_raster_files(root_dir):
    """
    Recursively search for .tif files in the root directory matching the pattern.
    Extracts the year (four digits) and crop type (four capital letters).
    Returns a dictionary of files categorized by crop type.
    """
    pattern = re.compile(r'_A_[a-zA-Z]{4}_A.tif$')
    file_dict = {}
    
    search_pattern = os.path.join(root_dir, '**', '*.tif')
    all_files = glob.glob(search_pattern, recursive=True)
    print(f"Number of .tif files found: {len(all_files)}")

    for file_path in all_files:
        file_name = os.path.basename(file_path)
        if pattern.search(file_name):
            year_match = re.search(r'(\d{4})', file_name)
            crop_type_match = re.search(r'_A_([A-Z]{4})_A', file_name)
            if year_match and crop_type_match:
                crop_type = crop_type_match.group(1)
                if crop_type not in file_dict:
                    file_dict[crop_type] = []
                file_dict[crop_type].append(file_path)
                
    print(f"Total matched crops: {len(file_dict)}")
    return file_dict


def preprocess(ds):
    """
    Preprocess each dataset to ensure consistent coordinate ordering.
    """
    # Ensure latitude is sorted in descending order
    if not np.all(ds['y'].values[:-1] >= ds['y'].values[1:]):
        ds = ds.sortby('y', ascending=False)
    # Ensure longitude is sorted in ascending order
    if not np.all(ds['x'].values[:-1] <= ds['x'].values[1:]):
        ds = ds.sortby('x', ascending=True)
    return ds


def open_crops_by_year(file_list):
    """
    Opens raster files by crop type using xarray.
    Combines yearly files into a single xarray dataset for each crop type.
    """
    ds = xr.open_mfdataset(
        file_list, 
        combine='by_coords', 
        preprocess=preprocess, 
        engine='rasterio'
    )
    return ds


# Usage example
root_directory = '/Users/mbraaksma/Files/base_data/spam'
raster_files = find_raster_files(root_directory)

# Open dataset for the crop type 'VEGE'
if 'VEGE' in raster_files:
    crop_dataset = open_crops_by_year(raster_files['VEGE'])
    print(crop_dataset)
else:
    print("No dataset found for 'VEGE'.")


Number of .tif files found: 3180
Total matched crops: 48


ValueError: Resulting object does not have monotonic global indexes along dimension y

NEXT STEP: RECLASSIFICATION TO 2000 TYPES OF CROPS. 

In [ ]:
import glob
import re
import os
import xarray as xr
import numpy as np

def find_raster_files(root_dir):
    """
    Recursively search for .tif files in the root directory matching the pattern.
    Extracts the year (four digits) and crop type (four capital letters).
    Returns a dictionary of files categorized by year.
    """
    # Pattern to match files with the format: 'A_CROP_A.tif' or '_P_CROP_A.tif' for 2000
    pattern = re.compile(r'(A_[A-Z]{4}_A_.tif$)|((?=.*2000).*(P_[A-Z]{4}_A.tif$))')
    file_dict = {}
    
    search_pattern = os.path.join(root_dir, '**', '*.tif')
    all_files = glob.glob(search_pattern, recursive=True)
    print(f"Number of .tif files found: {len(all_files)}")

    for file_path in all_files:
        file_name = os.path.basename(file_path)
        if pattern.search(file_name):
            # Extract the year and crop type
            year_match = re.search(r'(\d{4})', file_name)
            # crop_type_match = re.search(r'([A-Z]{4})', file_name)
            
            if year_match:
            # if year_match and crop_type_match:
                year = year_match.group(1)
                # Group files by year
                if year not in file_dict:
                    file_dict[year] = []
                    print(year)
                file_dict[year].append(file_path)
    
    print(f"Total years found: {len(file_dict)}")
    return file_dict


def preprocess(ds):
    """
    Preprocess each dataset to ensure consistent coordinate ordering.
    """
    # Ensure latitude is sorted in descending order
    if not np.all(ds['y'].values[:-1] >= ds['y'].values[1:]):
        ds = ds.sortby('y', ascending=False)
    # Ensure longitude is sorted in ascending order
    if not np.all(ds['x'].values[:-1] <= ds['x'].values[1:]):
        ds = ds.sortby('x', ascending=True)
    return ds


def open_crops_by_year(file_list):
    """
    Opens raster files by year using xarray.
    Combines files into a single xarray dataset for each year.
    """
    ds = xr.open_mfdataset(
        file_list, 
        combine='by_coords', 
        preprocess=preprocess, 
        engine='rasterio'
    )
    return ds


# Usage example
root_directory = '/Users/mbraaksma/Files/base_data/spam'
raster_files_by_year = find_raster_files(root_directory)

# Example: Open dataset for the year '2020'
# if '2020' in raster_files_by_year:
#     dataset_2020 = open_crops_by_year(raster_files_by_year['2020'])
#     print(dataset_2020)
# else:
#     print("No dataset found for the year '2020'.")


Number of .tif files found: 3180
2000
Total years found: 1


In [53]:
import glob
import re
import os
import xarray as xr
import numpy as np

def find_raster_files(root_dir):
    """
    Recursively search for .tif files in the root directory.
    Extracts the year (four digits) and crop type (four capital letters).
    Returns a dictionary of files categorized by year.
    """
    # Patterns for different file types
    pattern_A = re.compile(r'A_[A-Z]{4}_A\.tif$')
    pattern_P = re.compile(r'P_[A-Z]{4}_A\.tif$')
    
    file_dict = {}
    
    search_pattern = os.path.join(root_dir, '**', '*.tif')
    all_files = glob.glob(search_pattern, recursive=True)
    print(f"Number of .tif files found: {len(all_files)}")

    for file_path in all_files:
        file_name = os.path.basename(file_path)
        
        # Check for 'A_CROP_A.tif' pattern
        if pattern_A.search(file_name):
            year_match = re.search(r'(\d{4})', file_name)
            if year_match:
                year = year_match.group(1)
                if year not in file_dict:
                    file_dict[year] = []
                file_dict[year].append(file_path)
        
        # Check for 'P_CROP_A.tif' pattern AND '2000' in filename
        elif pattern_P.search(file_name) and '2000' in file_name:
            year_match = re.search(r'(\d{4})', file_name)
            if year_match:
                year = year_match.group(1)
                if year not in file_dict:
                    file_dict[year] = []
                file_dict[year].append(file_path)
    
    print(f"Total years found: {len(file_dict)}")
    return file_dict

def preprocess(ds):
    """
    Preprocess each dataset to ensure consistent coordinate ordering.
    """
    # Ensure latitude is sorted in descending order
    if not np.all(ds['y'].values[:-1] >= ds['y'].values[1:]):
        ds = ds.sortby('y', ascending=False)
    # Ensure longitude is sorted in ascending order
    if not np.all(ds['x'].values[:-1] <= ds['x'].values[1:]):
        ds = ds.sortby('x', ascending=True)
    return ds


def open_crops_by_year(file_list):
    """
    Opens raster files by year using xarray.
    Combines files into a single xarray dataset for each year.
    """
    ds = xr.open_mfdataset(
        file_list, 
        combine='by_coords', 
        preprocess=preprocess, 
        engine='rasterio'
    )
    return ds

# Usage example
root_directory = '/Users/mbraaksma/Files/base_data/spam'
raster_files = find_raster_files(root_directory)

print(raster_files['2020'])

# Example: Open dataset for the year '2020'
if '2020' in raster_files:
    dataset_2020 = open_crops_by_year(raster_files['2020'])
    print(dataset_2020)
else:
    print("No dataset found for the year '2020'.")


Number of .tif files found: 3180
Total years found: 4
['/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_VEGE_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_REST_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_SUGC_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_SORG_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_MAIZ_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_BANA_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_SWPO_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_phy

ValueError: Resulting object does not have monotonic global indexes along dimension y

In [2]:
import glob
import re
import os
import xarray as xr
import numpy as np

def find_raster_files(root_dir):
    """
    Recursively search for .tif files in the root directory.
    Extracts the year (four digits) and crop type (four capital letters).
    Returns a dictionary of files categorized by year.
    """
    pattern_A = re.compile(r'A_[A-Z]{4}_A\.tif$')
    pattern_P = re.compile(r'P_[A-Z]{4}_A\.tif$')
    
    file_dict = {}
    
    search_pattern = os.path.join(root_dir, '**', '*.tif')
    all_files = glob.glob(search_pattern, recursive=True)
    print(f"Number of .tif files found: {len(all_files)}")

    for file_path in all_files:
        file_name = os.path.basename(file_path)
        
        # Check for 'A_CROP_A.tif' pattern
        if pattern_A.search(file_name):
            year_match = re.search(r'(\d{4})', file_name)
            if year_match:
                year = year_match.group(1)
                if year not in file_dict:
                    file_dict[year] = []
                file_dict[year].append(file_path)
        
        # Check for 'P_CROP_A.tif' pattern AND '2000' in filename
        elif pattern_P.search(file_name) and '2000' in file_name:
            year_match = re.search(r'(\d{4})', file_name)
            if year_match:
                year = year_match.group(1)
                if year not in file_dict:
                    file_dict[year] = []
                file_dict[year].append(file_path)
    
    print(f"Total years found: {len(file_dict)}")
    return file_dict

def preprocess(ds, year, crop_type):
    """
    Preprocess each dataset to ensure consistent coordinate ordering.
    Adds 'time' and 'crop_type' dimensions.
    """
    # Ensure latitude is sorted in descending order
    if not np.all(ds['y'].values[:-1] >= ds['y'].values[1:]):
        ds = ds.sortby('y', ascending=False)
    # Ensure longitude is sorted in ascending order
    if not np.all(ds['x'].values[:-1] <= ds['x'].values[1:]):
        ds = ds.sortby('x', ascending=True)
    
    # Add 'time' and 'crop_type' as new coordinates
    ds = ds.expand_dims({'time': [int(year)], 'crop_type': [crop_type]})
    return ds

def open_crops_by_year(file_list, year):
    """
    Opens raster files by year using xarray.
    Combines files into a single xarray dataset for each year.
    """
    datasets = []
    for file_path in file_list:
        crop_type_match = re.search(r'_[A-Z]{4}_', os.path.basename(file_path))
        crop_type = crop_type_match.group(0).strip('_') if crop_type_match else 'UNKNOWN'
        
        ds = xr.open_dataset(file_path, engine='rasterio')
        ds = preprocess(ds, year, crop_type)
        datasets.append(ds)
    
    combined_ds = xr.combine_by_coords(datasets, combine_attrs='override')
    return combined_ds

# Usage example
root_directory = '/Users/mbraaksma/Files/base_data/spam'
raster_files = find_raster_files(root_directory)

print(raster_files['2000'])


Number of .tif files found: 3180
Total years found: 4
['/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2000v3.0.7_global_physical-area.geotiff/spam2000V3r107_global_P_MAIZ_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2000v3.0.7_global_physical-area.geotiff/spam2000V3r107_global_P_OTHE_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2000v3.0.7_global_physical-area.geotiff/spam2000V3r107_global_P_SUGC_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2000v3.0.7_global_physical-area.geotiff/spam2000V3r107_global_P_SORG_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2000v3.0.7_global_physical-area.geotiff/spam2000V3r107_global_P_GROU_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2000v3.0.7_global_physical-area.geotiff/spam2000V3r107_global_P_COTT_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2000v3.0.7_global_physical-area.geotiff/spam2000V3r107_global_P_OFIB_A.tif', 

In [3]:
%%time 

# Example: Open dataset for the year '2020'
if '2000' in raster_files:
    dataset_2020 = open_crops_by_year(raster_files['2000'], '2000')
    print(dataset_2020)
else:
    print("No dataset found for the year '2000'.")

<xarray.Dataset>
Dimensions:      (time: 1, crop_type: 21, band: 1, x: 4320, y: 2160)
Coordinates:
  * time         (time) int64 2000
  * crop_type    (crop_type) object 'BANP' 'BARL' 'BEAN' ... 'SWPY' 'WHEA'
  * band         (band) int64 1
  * x            (x) float64 -180.0 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * y            (y) float64 89.96 89.88 89.79 89.71 ... -89.79 -89.87 -89.96
    spatial_ref  int64 0
Data variables:
    band_data    (time, crop_type, band, y, x) float32 nan nan nan ... nan nan
CPU times: user 613 ms, sys: 493 ms, total: 1.11 s
Wall time: 1.2 s


In [4]:
%%time 

# Example: Open dataset for the year '2020'
if '2005' in raster_files:
    dataset = open_crops_by_year(raster_files['2005'], '2005')
    print(dataset)
else:
    print("No dataset found for the year '2005'.")

<xarray.Dataset>
Dimensions:      (time: 1, crop_type: 42, band: 1, x: 4320, y: 1853)
Coordinates:
  * time         (time) int64 2005
  * crop_type    (crop_type) object 'ACOF' 'BANA' 'BARL' ... 'WHEA' 'YAMS'
  * band         (band) int64 1
  * x            (x) float64 -180.0 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * y            (y) float64 89.96 89.88 89.79 89.71 ... -64.21 -64.29 -64.37
    spatial_ref  int64 0
Data variables:
    band_data    (time, crop_type, band, y, x) float32 nan nan nan ... nan nan
CPU times: user 4.81 s, sys: 3.48 s, total: 8.29 s
Wall time: 9.09 s


In [7]:
%%time

import glob
import re
import os
import xarray as xr
import numpy as np
import dask
from dask.diagnostics import ProgressBar

def find_raster_files(root_dir):
    """
    Recursively search for .tif files in the root directory.
    Extracts the year (four digits) and crop type (four capital letters).
    Returns a dictionary of files categorized by year.
    """
    pattern_A = re.compile(r'A_[A-Z]{4}_A\.tif$')
    pattern_P = re.compile(r'P_[A-Z]{4}_A\.tif$')
    
    file_dict = {}
    
    search_pattern = os.path.join(root_dir, '**', '*.tif')
    all_files = glob.glob(search_pattern, recursive=True)
    print(f"Number of .tif files found: {len(all_files)}")

    for file_path in all_files:
        file_name = os.path.basename(file_path)
        
        if pattern_A.search(file_name):
            year_match = re.search(r'(\d{4})', file_name)
            if year_match:
                year = year_match.group(1)
                if year not in file_dict:
                    file_dict[year] = []
                file_dict[year].append(file_path)
        
        elif pattern_P.search(file_name) and '2000' in file_name:
            year_match = re.search(r'(\d{4})', file_name)
            if year_match:
                year = year_match.group(1)
                if year not in file_dict:
                    file_dict[year] = []
                file_dict[year].append(file_path)
    
    print(f"Total years found: {len(file_dict)}")
    return file_dict

def preprocess(ds, year, crop_type):
    """
    Preprocess each dataset to ensure consistent coordinate ordering.
    Adds 'time' and 'crop_type' dimensions.
    """
    # Ensure latitude is sorted in descending order
    if not np.all(ds['y'].values[:-1] >= ds['y'].values[1:]):
        ds = ds.sortby('y', ascending=False)
    # Ensure longitude is sorted in ascending order
    if not np.all(ds['x'].values[:-1] <= ds['x'].values[1:]):
        ds = ds.sortby('x', ascending=True)
    
    # Add 'time' and 'crop_type' as new coordinates
    ds = ds.expand_dims({'time': [int(year)], 'crop_type': [crop_type]})
    return ds

def open_crops_by_year(file_list, year):
    """
    Opens and preprocesses raster files by year using Dask.
    Combines files into a single xarray dataset for each year.
    """
    datasets = []
    for file_path in file_list:
        try:
            crop_type_match = re.search(r'_[A-Z]{4}_', os.path.basename(file_path))
            crop_type = crop_type_match.group(0).strip('_') if crop_type_match else 'UNKNOWN'
            
            # Open dataset using Dask
            ds = xr.open_dataset(file_path, engine='rasterio', chunks={'x': 1000, 'y': 1000})
            ds = preprocess(ds, year, crop_type)
            datasets.append(ds)
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

    # Combine datasets using Dask
    combined_ds = xr.combine_by_coords(datasets, combine_attrs='override')
    return combined_ds

# Usage example
root_directory = '/Users/mbraaksma/Files/base_data/spam'
raster_files = find_raster_files(root_directory)

print(raster_files['2020'])

# Example: Open dataset for the year '2020' with Dask
if '2010' in raster_files:
    with ProgressBar():
        dataset = open_crops_by_year(raster_files['2010'], '2010')
        print(dataset)
else:
    print("No dataset found for the year '2010'.")


Number of .tif files found: 3180
Total years found: 4
['/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_VEGE_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_REST_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_SUGC_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_SORG_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_MAIZ_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_BANA_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_physical_area/spam2020_v1r0_global_A_SWPO_A.tif', '/Users/mbraaksma/Files/base_data/spam/Global_Geotiff/spam2020V1r0_global_phy

In [9]:
if '2020' in raster_files:
    with ProgressBar():
        dataset = open_crops_by_year(raster_files['2020'], '2020')
        print(dataset)
else:
    print("No dataset found for the year '2020'.")

ValueError: Resulting object does not have monotonic global indexes along dimension y

In [10]:
from dask.diagnostics import ProgressBar

def open_crops_by_year(file_list, year):
    """
    Opens and preprocesses raster files by year using Dask.
    Combines files into a single xarray dataset for each year.
    """
    datasets = []
    for file_path in file_list:
        try:
            crop_type_match = re.search(r'_[A-Z]{4}_', os.path.basename(file_path))
            crop_type = crop_type_match.group(0).strip('_') if crop_type_match else 'UNKNOWN'
            
            # Open dataset using Dask
            ds = xr.open_dataset(file_path, engine='rasterio', chunks={'x': 1000, 'y': 1000})
            ds = preprocess(ds, year, crop_type)
            datasets.append(ds)
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

    # Combine datasets using Dask
    combined_ds = xr.combine_by_coords(datasets, combine_attrs='override')
    
    # Trigger computation to show progress
    with ProgressBar():
        combined_ds.load()  # Load data into memory, showing progress
    
    return combined_ds

# Example: Open dataset for the year '2020' with Dask
if '2010' in raster_files:
    with ProgressBar():
        dataset = open_crops_by_year(raster_files['2010'], '2010')
        print(dataset)
else:
    print("No dataset found for the year '2010'.")


[########################################] | 100% Completed | 2.27 sms
[########################################] | 100% Completed | 2.37 s
<xarray.Dataset>
Dimensions:      (time: 1, crop_type: 42, band: 1, x: 4320, y: 2160)
Coordinates:
  * time         (time) int64 2010
  * crop_type    (crop_type) object 'ACOF' 'BANA' 'BARL' ... 'WHEA' 'YAMS'
  * band         (band) int64 1
  * x            (x) float64 -180.0 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * y            (y) float64 89.96 89.88 89.79 89.71 ... -89.79 -89.87 -89.96
    spatial_ref  int64 0
Data variables:
    band_data    (time, crop_type, band, y, x) float32 nan nan nan ... nan nan
